In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors 

df_airbnb = pd.read_csv('Airbnb_Open_Data.csv')

/var/folders/5v/s327w09x2b7bc679clc46r0h0000gq/T/ipykernel_69269/1412995961.py:6: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df_airbnb = pd.read_csv('Airbnb_Open_Data.csv')


In [19]:
# drop the column we do not use
df_airbnb = df_airbnb.drop(['id', 'host id','country','country code','license','last review'], axis=1)

# change host_identity_verified to binary int
df_airbnb['host_identity_verified'] = df_airbnb['host_identity_verified'].map({'verified': 1, 'unconfirmed': 0})

# combine the same group for neighbourhood group
df_airbnb['neighbourhood group'] = df_airbnb['neighbourhood group'].replace('brookln', 'Brooklyn').replace('manhatan', 'Manhattan')

# Fill missing values in the 'neighbourhood group' column with a placeholder (e.g., 'Unknown')
df_airbnb['neighbourhood group'].fillna('Unknown', inplace=True)
df_airbnb['neighbourhood'].fillna('Unknown', inplace=True)

# Apply the function to fill missing prices
df_airbnb['price'] = df_airbnb['price'].replace('[\$,]', '', regex=True)
df_airbnb['service fee'] = df_airbnb['service fee'].replace('[\$,]', '', regex=True)

# Convert the "price" column to a float data type
df_airbnb['price'] = df_airbnb['price'].astype(float)
df_airbnb['service fee'] = df_airbnb['service fee'].astype(float)

# Calculate the average price for each neighborhood
average_prices = df_airbnb.groupby('neighbourhood')['price'].mean()

# Function to fill missing prices with the average price of the neighborhood
def fill_missing_price(row):
    if pd.isnull(row['price']):
        return average_prices[row['neighbourhood']]
    else:
        return row['price']

# Apply the function to update missing prices
df_airbnb['price'] = df_airbnb.apply(fill_missing_price, axis=1)

#Apply latitude and longitude restrictions for NYC
df_airbnb = df_airbnb[(df_airbnb['lat']>40.55) &(df_airbnb['lat']<40.94) & (df_airbnb['long']>-74.1) & (df_airbnb['long']<-73.67)]


In [20]:
df_airbnb.isnull().sum()

NAME                                250
host_identity_verified              288
host name                           403
neighbourhood group                   0
neighbourhood                         0
lat                                   0
long                                  0
instant_bookable                    105
cancellation_policy                  76
room type                             0
Construction year                   213
price                                 0
service fee                         271
minimum nights                      408
number of reviews                   183
reviews per month                 15829
review rate number                  326
calculated host listings count      319
availability 365                    448
house_rules                       51925
dtype: int64

Bedford-Stuyvesant    7937
Williamsburg          7775
Harlem                5466
Bushwick              4982
Hell's Kitchen        3965
                      ... 
West Farms               6
Co-op City               5
Gerritsen Beach          3
Fort Wadsworth           2
Glen Oaks                2
Name: neighbourhood, Length: 197, dtype: int64